In [1]:
from sql_queries import query_db, pprint_results

%load_ext autoreload
%autoreload 2

In [2]:
db_name = 'reuters.db'

# Inspecting the Reuters Dataset and Basic Relational Algebra

__select__

Write a query that is equivalent to the following relational algebra expressions.

$σ_{docid=10398-txt-earn}(frequency)$

Run your query against your local database and determine the number of records returned. 

In [3]:
query = '''
SELECT count(*) 
FROM (SELECT *
      FROM frequency
      WHERE docid = '10398_txt_earn') x;
'''
print query_db(db_name, query)

+----------+
| count(*) |
+----------+
|   138    |
+----------+


__select project__

$π_{term}(σ_{docid=10398-txt-earn-and-count=1}(frequency))$

In [4]:
query = '''
SELECT term 
FROM (SELECT *
      FROM frequency
      WHERE docid = '10398_txt_earn'
      AND count = 1)
'''

nested_query = '''
SELECT count(*)
FROM ({0})
'''.format(query)

print query_db(db_name, nested_query)

+----------+
| count(*) |
+----------+
|   110    |
+----------+


__Union__

$π_{term}(σ_{docid=10398-txt-earn-and-count=1}(frequency))$ U $π_{term}(σ_{docid=925-txt-trade-and-count=1}(frequency))$

In [5]:
query1 = '''
SELECT term 
FROM (SELECT *
      FROM frequency
      WHERE docid = '10398_txt_earn'
      AND count = 1)
'''

query2 = '''
SELECT term 
FROM (SELECT *
      FROM frequency
      WHERE docid = '925_txt_trade'
      AND count = 1)
'''

union1 = '''
SELECT * 
FROM ({0} UNION {1})
'''.format(query1, query2)

nested_query = '''
SELECT count(*)
FROM ({0})
'''.format(union1)

print query_db(db_name, nested_query)

+----------+
| count(*) |
+----------+
|   324    |
+----------+


__Count__

Write a SQL statement to count the number of unique documents containing the word "law" or containing the word "legal

In [6]:
query = '''
SELECT DISTINCT docid
FROM frequency
WHERE term = 'law'
OR term = 'legal'
'''

nested_query = '''
SELECT count(*)
FROM ({0})
'''.format(query)

print query_db(db_name, nested_query)

+----------+
| count(*) |
+----------+
|    58    |
+----------+


__big documents__

Write a SQL statement to find all documents that have more than 300 total terms, including duplicate terms. 

(Hint: You can use the HAVING clause, or you can use a nested query. 

Another hint: Remember that the count column contains the term frequencies, and you want to consider duplicates.) (docid, term_count)

__** Use a group by to count the terms per document, then find those documents that have more than 300 terms.__

In [7]:
query1 = '''
SELECT docid
FROM frequency
GROUP BY docid
HAVING count(term) > 300
'''


nested_query = '''
SELECT count(*)
FROM ({0})
'''.format(query1)

print query_db(db_name, nested_query)

+----------+
| count(*) |
+----------+
|    11    |
+----------+


__ two words__

Write a SQL statement to count the number of unique documents that contain both the word 'transactions' and the word 'world'. 

(Hint: Find the docs that contain one word and the docs that contain the other word separately, then find the intersection.)

In [8]:
query1 = '''
SELECT *
FROM frequency
WHERE term = 'transactions'
'''

query2 = '''
SELECT *
FROM frequency
WHERE term = 'world'
'''

intersect = '''
SELECT docid 
FROM ({0})

INTERSECT 

SELECT docid 
FROM ({1})
'''.format(query1, query2)

nested_query = '''
SELECT count(*)
FROM ({0})
'''.format(intersect)

print query_db(db_name, nested_query)

+----------+
| count(*) |
+----------+
|    3     |
+----------+


---

# Matrix Multiplication in SQL

In [9]:
query1 = '''
SELECT * 
FROM a
'''
print query_db('matrix.db', query1)

+---------+---------+-------+
| row_num | col_num | value |
+---------+---------+-------+
|    0    |    3    |   55  |
|    0    |    4    |   78  |
|    1    |    0    |   19  |
|    1    |    2    |   21  |
|    1    |    3    |   3   |
|    1    |    4    |   81  |
|    2    |    1    |   48  |
|    2    |    2    |   50  |
|    2    |    3    |   1   |
|    3    |    2    |   33  |
|    3    |    4    |   67  |
|    4    |    0    |   95  |
|    4    |    4    |   31  |
+---------+---------+-------+


In [10]:
query1 = '''
SELECT * 
FROM b
'''
print query_db('matrix.db', query1)

+---------+---------+-------+
| row_num | col_num | value |
+---------+---------+-------+
|    0    |    1    |   73  |
|    0    |    4    |   42  |
|    1    |    2    |   82  |
|    2    |    0    |   83  |
|    2    |    1    |   13  |
|    2    |    3    |   57  |
|    3    |    0    |   48  |
|    3    |    1    |   85  |
|    3    |    2    |   18  |
|    3    |    3    |   24  |
|    4    |    0    |   98  |
|    4    |    1    |   7   |
|    4    |    4    |   3   |
+---------+---------+-------+


In [11]:
query1 = '''
SELECT a.row_num, b.col_num, a.col_num as j_a, b.row_num as j_b, a.value as a_val, b.value as b_val, a.value*b.value as cross_prod
FROM a, b
WHERE a.col_num = b.row_num
'''

print query_db('matrix.db', query1)

+---------+---------+-----+-----+-------+-------+------------+
| row_num | col_num | j_a | j_b | a_val | b_val | cross_prod |
+---------+---------+-----+-----+-------+-------+------------+
|    0    |    0    |  3  |  3  |   55  |   48  |    2640    |
|    0    |    1    |  3  |  3  |   55  |   85  |    4675    |
|    0    |    2    |  3  |  3  |   55  |   18  |    990     |
|    0    |    3    |  3  |  3  |   55  |   24  |    1320    |
|    0    |    0    |  4  |  4  |   78  |   98  |    7644    |
|    0    |    1    |  4  |  4  |   78  |   7   |    546     |
|    0    |    4    |  4  |  4  |   78  |   3   |    234     |
|    1    |    1    |  0  |  0  |   19  |   73  |    1387    |
|    1    |    4    |  0  |  0  |   19  |   42  |    798     |
|    1    |    0    |  2  |  2  |   21  |   83  |    1743    |
|    1    |    1    |  2  |  2  |   21  |   13  |    273     |
|    1    |    3    |  2  |  2  |   21  |   57  |    1197    |
|    1    |    0    |  3  |  3  |   3   |   48  |    14

In [12]:
query1 = '''
SELECT a.row_num, b.col_num, a.value*b.value as cross_prod
FROM a, b
WHERE a.col_num = b.row_num
'''

print query_db('matrix.db', query1)

+---------+---------+------------+
| row_num | col_num | cross_prod |
+---------+---------+------------+
|    0    |    0    |    2640    |
|    0    |    1    |    4675    |
|    0    |    2    |    990     |
|    0    |    3    |    1320    |
|    0    |    0    |    7644    |
|    0    |    1    |    546     |
|    0    |    4    |    234     |
|    1    |    1    |    1387    |
|    1    |    4    |    798     |
|    1    |    0    |    1743    |
|    1    |    1    |    273     |
|    1    |    3    |    1197    |
|    1    |    0    |    144     |
|    1    |    1    |    255     |
|    1    |    2    |     54     |
|    1    |    3    |     72     |
|    1    |    0    |    7938    |
|    1    |    1    |    567     |
|    1    |    4    |    243     |
|    2    |    2    |    3936    |
|    2    |    0    |    4150    |
|    2    |    1    |    650     |
|    2    |    3    |    2850    |
|    2    |    0    |     48     |
|    2    |    1    |     85     |
|    2    |    2    

In [13]:
query2 = '''
SELECT row_num, col_num, SUM(cross_prod)
FROM ({0})
GROUP BY row_num, col_num
'''.format(query1)

print query_db('matrix.db', query2)

+---------+---------+-----------------+
| row_num | col_num | SUM(cross_prod) |
+---------+---------+-----------------+
|    0    |    0    |      10284      |
|    0    |    1    |       5221      |
|    0    |    2    |       990       |
|    0    |    3    |       1320      |
|    0    |    4    |       234       |
|    1    |    0    |       9825      |
|    1    |    1    |       2482      |
|    1    |    2    |        54       |
|    1    |    3    |       1269      |
|    1    |    4    |       1041      |
|    2    |    0    |       4198      |
|    2    |    1    |       735       |
|    2    |    2    |       3954      |
|    2    |    3    |       2874      |
|    3    |    0    |       9305      |
|    3    |    1    |       898       |
|    3    |    3    |       1881      |
|    3    |    4    |       201       |
|    4    |    0    |       3038      |
|    4    |    1    |       7152      |
|    4    |    4    |       4083      |
+---------+---------+-----------------+


In [14]:
query3 = '''
SELECT row_num, col_num, SUM(cross_prod)
FROM ({0})
WHERE (row_num = 2 AND col_num = 3)
GROUP BY col_num, row_num
'''.format(query1)

print query_db('matrix.db', query3)

+---------+---------+-----------------+
| row_num | col_num | SUM(cross_prod) |
+---------+---------+-----------------+
|    2    |    3    |       2874      |
+---------+---------+-----------------+


# Working with a Term-Document Matrix

In [15]:
query1 = '''
SELECT * 
FROM frequency
LIMIT 10
'''

print query_db(db_name, query1)

+----------------+----------+-------+
|     docid      |   term   | count |
+----------------+----------+-------+
| 10000_txt_earn |   net    |   1   |
| 10000_txt_earn |  rogers  |   4   |
| 10000_txt_earn | earnings |   2   |
| 10000_txt_earn |  switch  |   1   |
| 10000_txt_earn |   conn   |   1   |
| 10000_txt_earn | revenues |   2   |
| 10000_txt_earn |   cts    |   1   |
| 10000_txt_earn | company  |   1   |
| 10000_txt_earn |   ago    |   1   |
| 10000_txt_earn | circuit  |   1   |
+----------------+----------+-------+


### h.) similarity matrix

In [16]:
query1 = '''
SELECT A.docid as A_id, B.docid as B_id, A.term as A_share_term,
B.term as B_share_term, a.count as A_count, b.count as B_count, A.count*B.count as cross_prod

FROM frequency as A, frequency as B
WHERE A.term = B.term 
AND A.docid < B.docid
LIMIT 10

'''

print query_db(db_name, query1)

+----------------+-----------------+--------------+--------------+---------+---------+------------+
|      A_id      |       B_id      | A_share_term | B_share_term | A_count | B_count | cross_prod |
+----------------+-----------------+--------------+--------------+---------+---------+------------+
| 10000_txt_earn | 10080_txt_crude |     net      |     net      |    1    |    1    |     1      |
| 10000_txt_earn |  10094_txt_earn |     net      |     net      |    1    |    3    |     3      |
| 10000_txt_earn |  10097_txt_earn |     net      |     net      |    1    |    4    |     4      |
| 10000_txt_earn |  1009_txt_earn  |     net      |     net      |    1    |    1    |     1      |
| 10000_txt_earn |  10114_txt_earn |     net      |     net      |    1    |    1    |     1      |
| 10000_txt_earn |  1011_txt_earn  |     net      |     net      |    1    |    6    |     6      |
| 10000_txt_earn |  10130_txt_earn |     net      |     net      |    1    |    3    |     3      |


In [17]:
query1 = '''
SELECT A.docid as A_id, B.docid as B_id, A.term as A_share_term,
B.term as B_share_term, a.count as A_count, b.count as B_count, A.count*B.count as cross_prod

FROM frequency as A, frequency as B
WHERE A.term = B.term 
AND A.docid < B.docid
AND (A.docid = '10080_txt_crude' AND B.docid = '17035_txt_earn')
'''

print query_db(db_name, query1)

+-----------------+----------------+--------------+--------------+---------+---------+------------+
|       A_id      |      B_id      | A_share_term | B_share_term | A_count | B_count | cross_prod |
+-----------------+----------------+--------------+--------------+---------+---------+------------+
| 10080_txt_crude | 17035_txt_earn |    april     |    april     |    1    |    2    |     2      |
| 10080_txt_crude | 17035_txt_earn |    ended     |    ended     |    1    |    1    |     1      |
| 10080_txt_crude | 17035_txt_earn |     inc      |     inc      |    1    |    1    |     1      |
| 10080_txt_crude | 17035_txt_earn |     mln      |     mln      |    2    |    3    |     6      |
| 10080_txt_crude | 17035_txt_earn |     net      |     net      |    1    |    3    |     3      |
| 10080_txt_crude | 17035_txt_earn |    profit    |    profit    |    1    |    4    |     4      |
| 10080_txt_crude | 17035_txt_earn |    reuter    |    reuter    |    1    |    1    |     1      |


In [18]:
query2 = '''
SELECT SUM(cross_prod)
FROM ({0})
'''.format(query1)

print query_db(db_name, query2)

+-----------------+
| SUM(cross_prod) |
+-----------------+
|        19       |
+-----------------+


### i.) keyword search

Add the query to the database as docid = 'q', and the terms of the query as terms.

We wont print this one out since it's a large database with these 3 records added.

In [19]:
query_add = '''
SELECT * FROM frequency
UNION
SELECT 'q' as docid, 'washington' as term, 1 as count 
UNION
SELECT 'q' as docid, 'taxes' as term, 1 as count
UNION 
SELECT 'q' as docid, 'treasury' as term, 1 as count
'''

# print query_db(db_name, query_add)

Now create a $DD^T$ similarity matrix using our updated database and filter by the query terms.

In [20]:
query_add_2 = '''
SELECT A.docid as A_id, B.docid as B_id, A.term as A_share_term,
B.term as B_share_term, a.count as A_count, b.count as B_count, A.count*B.count as cross_prod

FROM ({0}) as A, ({0}) as B
WHERE A.term = B.term 
AND A.docid < B.docid
AND B.docid = 'q'

ORDER BY cross_prod DESC

LIMIT 10

'''.format(query_add)

print query_db(db_name, query_add_2)

+--------------------+------+--------------+--------------+---------+---------+------------+
|        A_id        | B_id | A_share_term | B_share_term | A_count | B_count | cross_prod |
+--------------------+------+--------------+--------------+---------+---------+------------+
|  16094_txt_trade   |  q   |  washington  |  washington  |    6    |    1    |     6      |
|  16357_txt_trade   |  q   |  washington  |  washington  |    6    |    1    |     6      |
| 19775_txt_interest |  q   |   treasury   |   treasury   |    5    |    1    |     5      |
|  10623_txt_trade   |  q   |  washington  |  washington  |    5    |    1    |     5      |
| 12774_txt_interest |  q   |   treasury   |   treasury   |    4    |    1    |     4      |
|  233_txt_interest  |  q   |   treasury   |   treasury   |    4    |    1    |     4      |
|   5964_txt_trade   |  q   |   treasury   |   treasury   |    4    |    1    |     4      |
|  18399_txt_trade   |  q   |  washington  |  washington  |    4    | 

Now group the same documents together, and sum the total cross product of counts to include all of the terms.

Not much is added since not many documents contain multiple words from our query.

In [21]:
query_add_3 = '''
SELECT A_id, SUM(cross_prod) as total_cross_prod
FROM ({0})
GROUP BY A_id
ORDER BY total_cross_prod DESC
LIMIT 10
'''.format(query_add_2)

print query_db(db_name, query_add_3)

+--------------------+------------------+
|        A_id        | total_cross_prod |
+--------------------+------------------+
|  16094_txt_trade   |        6         |
|  16357_txt_trade   |        6         |
|  10623_txt_trade   |        5         |
| 19775_txt_interest |        5         |
| 12774_txt_interest |        4         |
|  18399_txt_trade   |        4         |
|  233_txt_interest  |        4         |
|   5964_txt_trade   |        4         |
| 16681_txt_interest |        3         |
| 18520_txt_interest |        3         |
+--------------------+------------------+
